**Rationale** Hypothesis testing is foundational to the entire field of statistics. Without the Central Limit Theorem and hypothesis testing, we would not have modern day (social) science. In marketing, this is generally most important when doing A/B testing of ads. In this assignment, you will practice computing basic summary statistics and conducting hypothesis tests.

[Datasets](https://drive.google.com/drive/folders/1kf6BHlmF32UjsHw6wh_sMJ-JWkV091XD?usp=sharing) required:
1. [FB ad campaign data](https://drive.google.com/file/d/1XqPgu_rFZZ5eHCoFS7drFsmqd7U81vj_/view?usp=sharing).
1. [Starbucks Promos](https://drive.google.com/file/d/1aUJUdGm-RdHaJCJvS1kay9fRdNTq22xb/view?usp=sharing)

# 1. (15 points) FB Ad campaigns

1. Use a groupby operation create a dataframe called `sumstats` consisting of the total impressions and clicks for each `xyz_campaign_id`.
1. Using `sumstats`, create a new column called `ctr` that represents the [click through rate](https://support.google.com/google-ads/answer/2615875?hl=en) for each campaign. Which campaign had the highest CTR?
1. Create a column for `sd` for standard deviation of the click through rate.

Hint: Recall from the notes that the standard deviation of a binomial distribution with $Pr(Success) = p$ is $SD(p) = \sqrt{p(1-p)}$. So if the probability of a binary outcome (such as clicking on an ad) is observed to be .3, the standard deviation is $\sqrt{.3(.7)}=\sqrt{.21}$.

In [ ]:
# imports and mount google drive
import os, pandas as pd, numpy as np
from scipy import stats

from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
# set the path to the datasets for A4
fpath = 'drive/My Drive/school/2023_fall/courses/suu/03-anly4100/assignments/04_assignment/data/'
os.listdir(fpath)

['facebook_ads.csv', 'starbucks_promos.csv']

In [ ]:
# read in the facebook_ads.csv file as the dataframe variable named fb
fb = pd.read_csv(fpath + 'facebook_ads.csv', index_col=0)

In [ ]:
# Take a look at the first 5 rows
# Observe that each row represents 1 ad from 1 campaign and a target demographic group (age, gender, interest)
fb.sort_values(by=['ad_id', 'xyz_campaign_id'])[['age', 'gender', 'interest']].head(5).reset_index()

,ad_id,age,gender,interest
0,708746,30-34,M,15
1,708749,30-34,M,16
2,708771,30-34,M,20
3,708815,30-34,M,28
4,708818,30-34,M,28


In [ ]:
# use a groupby to create new dataframe, sumstats that tabulates the total clicks and impressions for each campaign.
# Generic groupby syntax: sumstats = df.groupby('groupbyvariable')[['summary_variable1', 'summary_variable2']].summaryfunction().reset_index()
# use the correct dataframe name, variables (i.e. column names) and summaryfunction
# Hint: Analyze the different campaigns in 'xyz_campaign_id'
sumstats = fb.groupby('xyz_campaign_id')[['Impressions', 'Clicks']].sum().reset_index()
sumstats.head()

,xyz_campaign_id,Impressions,Clicks
0,916,482925,113
1,936,8128187,1984
2,1178,204823716,36068


In [ ]:
# create a new column in sumstats called 'ctr' (short for click through rate)
# recall that ctr = clicks / impressions

sumstats['ctr'] = sumstats['Clicks']/sumstats['Impressions']

In [ ]:
# take a look at the whole sumstats dataframe
# sumstats.head(6).reset_index()
sumstats.head()


,xyz_campaign_id,Impressions,Clicks,ctr
0,916,482925,113,0.000234
1,936,8128187,1984,0.000244
2,1178,204823716,36068,0.000176


In [ ]:
# create a new column ctr_sd for the std. dev of the ctr (See hint above)
# pay attention to PEMDAS, esp. that you want to take the sqrt after computing p*(1-p)
# np.sqrt(x) returns the sqrt of some number x
# x = sumstats['ctr'] * (1.0 - sumstats['ctr'])
# sumstats['ctr_sd']=np.sqrt(x)

sumstats['ctr_sd']=np.sqrt(sumstats['ctr'] * (1.0 - sumstats['ctr']))

In [ ]:
# take another look at the whole sumstats dataframe
# compare the ctr_std to calculating it by hand, make sure you did this correctly.
sumstats



,xyz_campaign_id,Impressions,Clicks,ctr,ctr_sd
0,916,482925,113,0.000234,0.015295
1,936,8128187,1984,0.000244,0.015621
2,1178,204823716,36068,0.000176,0.013269


In [ ]:
sumstats.loc[sumstats['xyz_campaign_id'] == 936, ['ctr', 'ctr_sd']]

,ctr,ctr_sd
1,0.000244,0.015621


In [ ]:
sumstats.iloc[0,3]

0.00023399078531863125

## Answer (edit this cell)
The campaign with the highest CTR is: **936**

# 2. (15 points) Hypothesis testing and confidence intervals

1. Compute the 95% confidence interval for the CTR for each campaign. Compare the confidence intervals of campaign 916 and 936. What can you conclude about the relative perfrmance of the 2 ads in the population (e.g. are they very different, similar, etc.)? How about 916 vs. 1178?
1. Was campaign 936 statistically different compared to campaign 916? How about 936 vs. 1178? Use the `ttest_2sample` function from the notes. Remember, you must define it in your Colab session in order to use it (execute the cell w/ the function).

Given these statistical tests what would you recommend in terms of allocation of the ad budget?

In [ ]:
# use stats.norm.ppf([.025, .975], mean, std error)
# where mean is sample mean and std error is std dev / sqrt(obs) (from Central Limit Theorem)

# confidence interval for ad 916
# ad_916 = stats.norm.ppf([.025, .975], sumstats[0]['ctr'], sumstats['ctr_sd']/np.sqrt(sumstats['Impressions']))
# ad_916 = stats.norm.ppf([.025, .975], sumstats.values[0][3], sumstats.values[0][4]/np.sqrt(sumstats.values[0][1]))

ad_916 = stats.norm.ppf([.025, .975], sumstats.values[0][3], sumstats.values[0][4]/np.sqrt(sumstats.values[0][1]))

print('The confidence interval for ad916 at 0.025 and 0.975 are {:.6f}, {:.6f}, respectively '.format(ad_916[0], ad_916[1]))


The confidence interval for ad916 at 0.025 and 0.975 are 0.000191, 0.000277, respectively 


In [ ]:
# confidence interval for ad 936
ad_936 = stats.norm.ppf([.025, .975], sumstats.values[1][3], sumstats.values[1][4]/np.sqrt(sumstats.values[1][1]))
print('The confidence interval for ad916 at 0.025 and 0.975 are {:.6f}, {:.6f}, respectively '.format(ad_936[0], ad_936[1]))

The confidence interval for ad916 at 0.025 and 0.975 are 0.000233, 0.000255, respectively 


In [ ]:
# confidence interval for ad 1178

ad_1178 = stats.norm.ppf([.025, .975], sumstats.values[2][3], sumstats.values[2][4]/np.sqrt(sumstats.values[2][1]))
print('The confidence interval for ad916 at 0.025 and 0.975 are {:.6f}, {:.6f}, respectively '.format(ad_1178[0], ad_1178[1]))

The confidence interval for ad916 at 0.025 and 0.975 are 0.000174, 0.000178, respectively 


**Edit this cell**

1. Based on sample sizes, the 936 campaign has a smaller bound than the other campaigns and thus, has a more accuracy on clicks per impressions.
1. 1178 campaign has a smaller bound than 916 and thus, It would be more accurate on an average of clicks per impressions.

In [ ]:
# Use this custom funciton

def ttest_2sample(m1,sd1,N1,m2,sd2,N2, twotail = True, equalvar = False):
    """
    This function requires you to supply:
    m1: mean of sample 1
    sd1: std. dev of sample 1
    N1: number of obs of sample 1
    m2: mean of sample 2
    sd2: std dev of sample 2
    N2: number of obs of sample 2

    Optional inputs:
    twotail = True (default) / False. If False, then 1 tail
    equalvar = True / False (default). If True, assumes equal population variance.
    """

    # The difference between equal and unequal variance is only in how to compute
    # the test statistic and degree of freedom.
    if equalvar:
        spsquare = ((N1-1)*sd1**2+(N2-1)*sd2**2)/(N1+N2-2)
        T = (m1-m2)/np.sqrt(spsquare*(1/N1+1/N2))
        nu = N1+N2-2
    else:
        nu = (sd1**2/N1+sd2**2/N2)**2/((sd1**2/N1)**2/(N1-1)+(sd2**2/N2)**2/(N2-1)) # new degree of freedom
        T = (m1-m2)/(np.sqrt(sd1**2/N1+sd2**2/N2))

    # If the first mean is bigger, we need to do 1- cdf
    # Otherwise just compute cdf
    if m1>m2:
        pval = 1-stats.t.cdf(T, df = nu)
    else:
        pval = stats.t.cdf(T, df = nu)

    # return p values
    # If 2 tail, we must multiply by 2
    # otherwise we just return the computed pval
    if twotail == True:
        return pval*2
    else:
        return pval

In [ ]:
# for each campaign grab the Pr(Success) = p which is ctr, std dev, and observations.
# p916 = float(sumstats.loc[sumstats.xyz_campaign_id==916, 'ctr'])
# sd916 = np.sqrt(p916*(1-p916))
# obs916 = float(sumstats.loc[sumstats.xyz_campaign_id==916, 'Impressions'])
p916 = float(sumstats.loc[sumstats.xyz_campaign_id==916, 'ctr'])
sd916 = np.sqrt(p916*(1-p916))
obs916 = float(sumstats.loc[sumstats.xyz_campaign_id==916, 'Impressions'])

#p936, sd936, obs936
p936 = float(sumstats.loc[sumstats.xyz_campaign_id==936, 'ctr'])
sd936 = np.sqrt(p936*(1-p936))
obs936 = float(sumstats.loc[sumstats.xyz_campaign_id==936, 'Impressions'])

#p1178, sd1178, obs1178
p1178 = float(sumstats.loc[sumstats.xyz_campaign_id==1178, 'ctr'])
sd1178 = np.sqrt(p1178*(1-p1178))
obs1178 = float(sumstats.loc[sumstats.xyz_campaign_id==1178, 'Impressions'])

# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 916
ttest_2sample(p936, sd936, obs936, p916, sd916, obs916, twotail = True, equalvar=False)




0.6561623087350004

In [ ]:
# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 1178
ttest_2sample(p936, sd936, obs936, p1178, sd1178, obs1178, twotail = True, equalvar=False)

0.0

**Edit this cell to answer**

(Write a few sentences to describe how to proceed with the allocation of the ad budget going forward. Be sure to use the 2 tail t-tests above to support your strategy)

For ad 916 and ad 1178, their p-values are greater than 0.05 and thus, we fail to reject the null hypothesis. Budget should be divided equally among the campaigns.

For ad 936 and ad 1178, the p-value are less than 0.05 and thus, we should reject the null hypothesis. The mean of these campaigns are not equal.

# 3. (20 points)

Using the Starbucks dataset, conduct an appropriate statistical test to determine:
1. Is there any statistically appreciable difference in the redemption rates of BOGO vs. Discount promotions?
1. Among those who redeemed an offer and reported their income, is there any statistical difference in the average income associated with BOGO vs. discount redemptions? Note that not every observation reports income (select only the observations that do).

In [ ]:
# read in the starbucks_promos.csv file as the dataframe variable named sb, use index_col = 0 option
sb = pd.read_csv(fpath + 'starbucks_promos.csv', index_col = 0)

In [ ]:
# preview first 5 rows to get a sense of the contents of the dataframe
# note that each observation represents an offer received by a customer
sb.head()


,uid,event,time,gender,age,register_date,income,offer_id,offer_reward,channels,difficulty,duration,offer_type,offer_time,transaction_amount,redeem_time,redeemed
1,0020c2b971eb4e9188eac86d93036a77,offer received,0,F,59,20160304,90000.0,fafdcd668e3743c1bb461111dcafc2a4,2.0,"['web', 'email', 'mobile', 'social']",10.0,240.0,discount,0.0,17.63,54.0,1
4,005500a7188546ff8a767329a2f7c76a,offer received,0,M,56,20171209,47000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,NaN,NaN,0
5,0056df74b63b4298809f0b375a304cf4,offer received,0,M,54,20160821,91000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"['web', 'email', 'mobile']",5.0,168.0,bogo,0.0,27.86,132.0,1
6,00715b6e55c3431cb56ff7307eb19675,offer received,0,F,58,20171207,119000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,27.26,12.0,1
8,00840a2ca5d2408e982d56544dc14ffd,offer received,0,M,26,20141221,61000.0,2906b810c7d4411798c6938adc9daaa5,2.0,"['web', 'email', 'mobile']",10.0,168.0,discount,0.0,6.05,540.0,1


In [ ]:
# select the redeemed column for discount offers types, name this data the variable disc
# ( e.g. disc = sb.loc[sb.offer_type=='discount', 'redeemed'] )
disc = sb.loc[sb.offer_type=='discount', 'redeemed']


In [ ]:
# select the redeemed column for buy one get one (bogo) offers, name this data the variable bogo
bogo = sb.loc[sb.offer_type=='bogo', 'redeemed']
# bogo = sb.loc[(sb.redeemed == 1) & (sb.offer_type=='bogo'), ['income']]



In [ ]:
# use stats.ttest_ind to test whether the 2 types of offers yield different redemption rates
stats.ttest_ind(bogo, disc, equal_var=False)


TtestResult(statistic=-17.301348973719907, pvalue=6.641774929557673e-67, df=61010.718998306074)

In [ ]:
# Select the income column for those who redeemed the BOGO offer, call this the variable bogo_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for bogo offer, and one for income not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data
# bogo_income = sb[(sb['redeemed']==True) & (sb['offer_type']=='bogo') & (sb.income.notnull())]
bogo_income = sb.loc[(sb.redeemed== 1) & (sb.offer_type == 'bogo'), ['income']]

bogo_income = bogo_income.loc[bogo_income['income'].notnull()]

In [ ]:
# Select the income column for those who redeemed the discount offer, call this the variable disc_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for discount offer,  and one for income not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data
# disc_income = sb[(sb['redeemed']==True) & (sb['offer_type']=='discount') & (sb.income.notnull())]
disc_income = sb.loc[(sb.redeemed== 1) & (sb.offer_type == 'discount'), ['income']]
disc_income = disc_income.loc[disc_income['income'].notnull()]


In [ ]:
# What is the average income for bogo redeemers (bogo_income.mean())?
bogo_income.mean()


income    70114.855959
dtype: float64

In [ ]:
# What is the average income for discount redeemers?
disc_income.mean()


income    68617.391304
dtype: float64

In [ ]:
bogo_income.mean() - disc_income.mean()

income    1497.464655
dtype: float64

In [ ]:
# use stats.ttest_ind to test whether the average income of the redeemers of the 2 offer types are different
stats.ttest_ind(bogo_income, disc_income, equal_var = False)


TtestResult(statistic=array([6.34291136]), pvalue=array([2.28357482e-10]), df=array([33258.93826493]))

**Edit this cell**
1. (Write a sentence to describe the relative redemption rates of the 2 offer types based on the result of the t-test).

Both of the p-values are lower than 0.05, even lower than 0.01, thus  we should reject the null hypothesis.

1. (Write a sentence to describe the relative incomes of the redeemers of the 2 offer types based on the result of the t-test).

There is a difference of about 1,500 USD in the relative incomes of the redeemers between the 2 offer types. The income mean of the bogo redeemers is sightly greater than the income mean of the disount redeemers.